In [1]:
include("../src/Utero.jl")
using .Utero 

In [2]:
ctx = ⬅Context()
ForwardBackward!(ctx, x -> 2.0*x + x, AddParams!(ctx, 7.0))

(21.0, Any[3.0])

In [3]:
# should be 12.8
ctx = ⬅Context()
ForwardBackward!(ctx, x -> x^2.0 + x^2.0, AddParams!(ctx, 3.2)) 

(20.480000000000004, Any[12.8])

In [4]:
# Gradient should be 56.21
ctx = Utero.⬅Context()
ForwardBackward!(ctx, w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), AddParams!(ctx, 7.0))

(265.0462473472405, Any[56.20829561241092])

In [5]:
# should be approx. (27.53, 3.0428)
ctx = Utero.⬅Context()
x, y = AddParams!(ctx, 13.2, 89.0)
ForwardBackward!(ctx, (x, y) -> (cos(y)^3 * 4sin(x))^2 + cos(x*y)^2, x, y) 

(1.0748674472530375, Any[27.53002464314667, 3.0428042273213602])

In [6]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W[5:5]' * W[5:5], 3)

(0.38768306858810453, Any[sparse([5], [1], [1.2452840135296117], 30, 1)])

In [7]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [8]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(30,1))
(out, grads) = ForwardBackward!(ctx, _ -> W' * W, 3)
print(size(grads[1]))

(30, 1)

In [9]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> W .* 2, nothing)
@show size(W.val)
print(size(grads[1]))
print(grads[1])


size(W.val) = (2, 1)
(2, 1)

[2.0; 2.0;;]

In [10]:
W = rand(2,1)
@show size(((W' * W) .* W)' * W)

U1, C1 = ⬅Dual(adjoint, W)
@show size(U1)
U2, C2 = ⬅Dual(*, U1, W)
@show size(U2)
U3, C3 = ⬅Dual(Base.broadcasted, *, U2, W)
@show size(U3)
U4, C4 = ⬅Dual(adjoint, U3)
@show size(U4)
U5, C5 = ⬅Dual(*, U4, W)
@show size(U5)

DU5 = [1]
@show size(DU5)
DU4, _ = C5(DU5)
@show size(DU4)
DU3 = C4(DU4)
@show size(DU3)
DU2, _ = C3(DU3)
@show size(DU2)
DU1, _ = C2(DU2)
@show size(DU1)




size(((W' * W) .* W)' * W) = (1, 1)
size(U1) = (1, 2)
size(U2) = (1, 1)
size(U3) = (2, 1)
size(U4) = (1, 2)
size(U5) = (1, 1)
size(DU5) = (1,)
size(DU4) = 

(1, 2)
size(DU3) = (2, 1)
size(DU2) = (1, 1)
size(DU1) = (1, 2)


(1, 2)

In [11]:
ctx = Utero.⬅Context()
W = AddParams!(ctx, rand(2,1))
(out, grads) = ForwardBackward!(ctx, _ -> ((W' * W) .* W)' * W, nothing)
@show size(W.val)
@show size(grads[1])

size(W.val) = (2, 1)
size(grads[1]) = (2, 1)


(2, 1)

In [12]:
Softmax(rand(2,1))

2×1 Matrix{Float64}:
 0.591213510356914
 0.40878648964308595

In [13]:
ctx = Utero.⬅Context()
W1, W2, W3 = AddParams!(ctx, rand(30,30), rand(40,30), rand(30,40))

function model(X)
    return  ReLU(W3 * ReLU(W2 * ReLU(W1 * X)))
end

X = rand(1500, 30)
Y = mapslices(row -> cos.(row), X, dims=2) 

println(ctx.Params[1].val)
for (x, y) in zip(eachrow(X), eachrow(Y))
    println(MeanSquaredError(model(x), y).val)
    (out, grads) = ForwardBackward!(ctx, x -> MeanSquaredError(model(x), y), x)
    GradientStep!(0.0001, ctx.Params, grads)
    exit("")
end
println(ctx.Params[1].val)


[0.8086024299806233 0.37389940475604044 0.4785833594218837 0.9910966038323754 0.5774544943009727 0.34005076666503387 0.2008400684450018 0.5828087683310229 0.22739947328646515 0.010275842462432738 0.4948631088783405 0.09649154667816429 0.5255872489111573 0.7269630665281062 0.9895138230130155 0.7943400445909786 0.2964216404745643 0.3599099074217281 0.11666027115231159 0.8446315761348103 0.9288099508663064 0.2284841345605848 0.7224046427391004 0.02077056788824949 0.6140742162123158 0.17164006006934707 0.780338162186656 0.45312881221374146 0.30277060654174404 0.9988860884877802; 0.1099204727365879 0.02096363299335202 0.6163550257441429 0.1777127196318169 0.9569907902443185 0.8170322079825317 0.06642586744159762 0.7201841776143244 0.8589929299553021 0.722009002903628 0.26682224581950464 0.3319777838234925 0.19138667519648767 0.6330508305966883 0.60585326310083 0.34181579436336273 0.46196550305436346 0.4985056757234583 0.6270201913612435 0.7754819589483057 0.8857345200381893 0.42909692936245

 0.747214593334849 0.35160320445911086 0.47208670835338995 0.3761119097999187 0.1425694407854794 0.45754295593017136 0.641674650826725 0.7203025600371235 0.8435165770788726; 0.14142052079368406 0.11885424739331607 0.48180904014352843 0.8179572050101139 0.09649444331365509 0.11904667572292682 0.5539187225144669 0.8124506398581379 0.9438078510425002 0.18660203565223876 0.4186560271420644 0.9995578083517022 0.5418249910017916 0.8514223972634124 0.5669894328859957 0.4406586155206229 0.8374164696269545 0.9012785743250009 0.018584790444112764 0.8194848530910097 0.19295401228088016 0.5392755374121818 0.8130357631017147 0.15419836470380321 0.19143050362576775 0.93815954330109 0.6552650418381034 0.17553250843543178 0.8689927994546599 0.653126568828559; 0.28054253211590363 0.5019688279669754 0.26745832556443727 0.04941113388558471 0.4709909273156865 0.8630134010835285 0.03471040006450976 0.5115988630548354 0.017762476918827153 0.9799818368498999 0.019095108743746536 0.36628606841954703 0.2711599

1.5193356934698272e8


MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int32

Closest candidates are:
  convert(::Type{T}, !Matched::Ptr) where T<:Integer
   @ Base pointer.jl:23
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


In [14]:
using MLDatasets

TrainData = MNIST(:train)
ValidData = MNIST(:test)

ctx = ⬅Context()

# Create Network
densenet = Sequential(
    Dense(28*28, 16, ReLU), 
    Dense(16, 16, ReLU), 
    Dense(16, 10, ReLU), 
    Softmax)
AddParams!(ctx, densenet)

# Perform gradient steps
for (x, y) in DataIterator(TrainData.features, TrainData.targets)
    x = reshape(x, 28*28, 1)
    y = OneHot(Int64(y+1), 10)
    (out, paramgrads) = ForwardBackward!(ctx, x -> MeanSquaredError(densenet(x), y), x)
    GradientStep!(0.0001, ctx.Params, paramgrads)
    exit()
end

Main.Utero.⬅Tracker{Matrix{Float64}}([-1.023291785234263 1.0081925561234168 … -1.0054180698260746 0.1476965414759338; -1.5745539091224192 0.3325675215377671 … -0.4846788074439304 0.1068658439216784; … ; -1.1292663433123287 0.28645897308587015 … 0.9084637622975797 -1.3141531892026836; 0.02429633909434002 -0.0706855256952554 … -0.6871583274472971 2.0894341516962864], UUID("0027538e-c898-11ee-3333-8bfb92686eef"), Any[], Any[])

Main.Utero.⬅Tracker{Matrix{Float64}}


MethodError: MethodError: no method matching length(::Main.Utero.⬅Tracker{Matrix{Float64}})

Closest candidates are:
  length(!Matched::Union{Base.KeySet, Base.ValueIterator})
   @ Base abstractdict.jl:58
  length(!Matched::Union{SparseArrays.FixedSparseVector{Tv, Ti}, SparseArrays.SparseVector{Tv, Ti}} where {Tv, Ti})
   @ SparseArrays /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/SparseArrays/src/sparsevector.jl:95
  length(!Matched::Union{Tables.AbstractColumns, Tables.AbstractRow})
   @ Tables ~/.julia/packages/Tables/AcRIE/src/Tables.jl:180
  ...
